In [1]:
from haystack.document_stores import PineconeDocumentStore

API_KEY = "<<YOUR_API_KEY>>"
INDEX_NAME = "haystack-nhs-jul"

document_store = PineconeDocumentStore(
    api_key=API_KEY,
    index=INDEX_NAME,
    embedding_dim=768
)

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.
INFO - haystack.document_stores.pinecone -  Index statistics: name: haystack-nhs-jul, embedding dimensions: 768, record count: 0


In [2]:
document_store.get_document_count(), document_store.get_embedding_count()

(100, 100)

In [3]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model='sentence-transformers/multi-qa-mpnet-base-dot-v1',
    model_format="sentence_transformers"
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.retriever.dense -  Init retriever using embeddings of model flax-sentence-embeddings/all_datasets_v3_mpnet-base


In [4]:
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

reader = FARMReader(
    model_name_or_path='deepset/roberta-base-squad2-distilled',
    use_gpu=True
)
pipe = ExtractiveQAPipeline(reader, retriever)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/electra-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/electra-base-squad2
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 9 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\  /w\ 
INFO - haystack.modeling.infer -  /'\   / \   /'\   /'\   / \   / \   /'\   /'\   /'\ 


Now we can begin asking questions:

In [5]:
prediction = pipe.run(
    query="Who is affected by pre-eclampsia?",
    params={
        "Retriever": {"top_k": 5},
        "Reader": {"top_k": 2}
    }
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless fil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabli

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/Users/jamesbriggs/opt/anaconda3/envs/ml/lib/python3.9/site-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.63 Batches/s]


In [6]:
from haystack.utils import print_answers
print_answers(prediction, details="minimum")


Query: Who is affected by pre-eclampsia?
Answers:
[   {   'answer': 'the mother and her baby',
        'context': 'cause no problems and improve soon after the baby is '
                   "delivered, there's a risk of serious complications that "
                   'can affect both the mother and her baby. '},
    {   'answer': 'before you were pregnant having an autoimmune condition, '
                  'such as lupus or antiphospholipid syndrome having high '
                  'blood pressure or pre-eclampsia in a previous pregnancy',
        'context': ' before you were pregnant having an autoimmune condition, '
                   'such as lupus or antiphospholipid syndrome having high '
                   'blood pressure or pre-eclampsia in a previous pregnancy'}]


We can see the top answer seems to be correct. To extract each component here rather than print with the built in method, we can do this:

In [14]:
prediction['answers'][0].context

"cause no problems and improve soon after the baby is delivered, there's a risk of serious complications that can affect both the mother and her baby. "

In [26]:
start = prediction['answers'][0].offsets_in_context[0].start
end = prediction['answers'][0].offsets_in_context[0].end

start, end

(125, 148)

In [27]:
prediction['answers'][0].context[start:end]

'the mother and her baby'

In [28]:
prediction['answers'][0]

<Answer {'answer': 'the mother and her baby', 'type': 'extractive', 'score': 0.9425204694271088, 'context': "cause no problems and improve soon after the baby is delivered, there's a risk of serious complications that can affect both the mother and her baby. ", 'offsets_in_document': [{'start': 631, 'end': 654}], 'offsets_in_context': [{'start': 125, 'end': 148}], 'document_id': 'f65f2a23620b9ea4761c43683729fc6c', 'meta': {'url': 'www.nhs.uk_conditions_pre-eclampsia_.txt'}}>

In [29]:
prediction['answers'][0].score

0.9425204694271088

In [31]:
prediction['answers'][0].meta['url']

'www.nhs.uk_conditions_pre-eclampsia_.txt'

---